In [1]:
import sys
sys.path.insert(0,'..')
from gp3.inference.mfsvi import MFSVI
from gp3.inference.laplace import Laplace
from gp3.likelihoods.likelihoods import Poisson
from gp3.utils import data as sim
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
from IPython.display import display
init_notebook_mode(connected=True)
import GPy
import warnings
warnings.filterwarnings('ignore')
from tqdm import trange
import numpy as np
from lif import LIFLike, LIFSim

In [2]:
X = sim.sim_X_equispaced(D = 2, N_dim = 20, lower=0, upper=100)
f = sim.sim_f(X, k=GPy.kern.RBF(2, variance=1.0, lengthscale=40.), mu = 3.0)

lif_gen = LIFSim()
spikes = lif_gen.sim(f)

trace_func = go.Scatter3d(x = X[:,0], y = X[:,1], z=f, mode = 'markers', marker=dict(size = 2,), name = "cell shape")
trace_draws = go.Scatter3d(x = X[:,0], y = X[:,1], z=spikes, mode = 'markers', marker=dict(size = 2,), name = "spike times")
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_func, 1, 1)
fig.append_trace(trace_draws, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



In [3]:
mu = np.ones(X.shape[0])*3.

inf_lp = Laplace(mu, GPy.kern.RBF(1, 1., 40.), LIFLike(ts = spikes.astype(np.int32)), X, spikes.astype(np.int32), verbose = True)
inf_svi = MFSVI(GPy.kern.RBF(1, 1., 40.), LIFLike(ts = spikes.astype(np.int32)), X, spikes.astype(np.int32), mu, verbose = True, max_grad = 1.)
inf_lp.run(10)
inf_svi.run(1000)

Objective: 123.20 | Step Size: 1.00: 100%|██████████| 10/10 [00:00<00:00, 50.03it/s]
ELBO: -490.80 | KL: 362.53 | logL: -128.27 | step: 0.0009765625: 100%|██████████| 1000/1000 [00:11<00:00, 84.99it/s]


In [4]:
trace_lppred = go.Scatter3d(x = X[:,0], y = X[:,1], z=inf_lp.f_pred, mode = 'markers', marker=dict(size = 2,), name = "laplace prediction")
trace_svipred = go.Scatter3d(x = X[:,0], y = X[:,1], z=inf_svi.predict(), mode = 'markers', marker=dict(size = 2,), name = "svi prediction")
fig = tools.make_subplots(rows=1, cols=3, specs=[[{'is_3d': True}, {'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_func, 1, 1)
fig.append_trace(trace_lppred, 1, 2)
fig.append_trace(trace_svipred, 1, 3)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]  [ (1,3) scene3 ]



In [5]:
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=np.exp(inf_svi.q_S), mode = 'markers', marker=dict(size = 2,), name = "svi variances")])